In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import time
import glob
import numpy as np
from bagpy import bagreader
import rosbag
import pandas as pd
import wandb

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [3]:
wandb.util.generate_id()

'3h9hh99m'

In [4]:
group_id = '29ivurzi'
# run = wandb.init(project="screwing_estimation", entity="serialexperimentsleon", group=group_id)
# run.finish()

In [3]:
os.cpu_count()

24

In [35]:
from screwing_dataset import ScrewingDataset
from screwing_model import ScrewingModel, ScrewingModelSeq, ScrewingModelGaussian
# from screwing_model_seq import 


In [36]:
from torch.utils.data import DataLoader, ConcatDataset
import torch
import torch.nn as nn
import torch.optim as optim

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [38]:
base_dset_dir = os.path.expanduser('~/datasets/screwing')

# xprmnt_dir = time.strftime("/%Y-%m-%d_%H-%M-%S")
xprmnt_dir = time.strftime("/2022-04-14_15-49-01")
# xprmnt_dir = time.strftime("/test")


In [10]:
bag_path_names = base_dset_dir + xprmnt_dir + '/' + str(4) + '_*.bag' 
# print(bag_path_names)
bag_path = glob.glob(bag_path_names)[0]
print(bag_path)
# bag = rosbag.Bag(bag_path)
breader = bagreader(bag_path)
bag = rosbag.Bag(bag_path)

main_topic = '/panda/franka_state_controller_custom/franka_states'
main_num_msgs = bag.get_message_count(main_topic)

print(main_num_msgs)


/home/serialexperimentsleon/datasets/screwing/2022-04-14_15-49-01/4_2022-04-14-15-50-27.bag
[INFO]  Data folder /home/serialexperimentsleon/datasets/screwing/2022-04-14_15-49-01/4_2022-04-14-15-50-27 already exists. Not creating.
206


In [24]:
dset_list = []
window_size = 10
num_dset = 2
for i in range(num_dset):
    id_str = str(i)
    bag_path_names = base_dset_dir + xprmnt_dir + '/' + id_str + '_*.bag' 
    bag_path = glob.glob(bag_path_names)[0]
    print(bag_path)

    pos_path_name = base_dset_dir + xprmnt_dir + '/' + id_str + '_pos.npy'
    proj_ori_path = base_dset_dir + xprmnt_dir + '/' + id_str + '_proj_ori.npy'
    pos_ori_path_list = [pos_path_name, proj_ori_path]
    
    dset_list.append(ScrewingDataset(bag_path, pos_ori_path_list, window_size, overlapping=True))
    
concat_dset = ConcatDataset(dset_list)

/home/serialexperimentsleon/datasets/screwing/2022-04-14_15-49-01/0_2022-04-14-15-49-06.bag
[INFO]  Data folder /home/serialexperimentsleon/datasets/screwing/2022-04-14_15-49-01/0_2022-04-14-15-49-06 already exists. Not creating.
/home/serialexperimentsleon/datasets/screwing/2022-04-14_15-49-01/1_2022-04-14-15-49-58.bag
[INFO]  Data folder /home/serialexperimentsleon/datasets/screwing/2022-04-14_15-49-01/1_2022-04-14-15-49-58 already exists. Not creating.


In [25]:
batch_size = 2

train_lder = DataLoader(
    concat_dset,
    shuffle=True,
    num_workers=2,
    batch_size=batch_size
)

# test_lder = DataLoader(
#     test_dset,
#     shuffle=True,
#     num_workers=2,
#     batch_size=batch_size
# )

In [39]:
input_dim = 19
hidden_dim = 10
num_layers = 3
output_dim = 5
model = ScrewingModelGaussian(input_dim, hidden_dim, num_layers, output_dim)
model.to(device)

ScrewingModelGaussian(
  (lstm): LSTM(19, 10, num_layers=3, batch_first=True)
  (hidden2out): Linear(in_features=10, out_features=10, bias=True)
)

In [40]:
from training import GNLL_loss

In [58]:
ori_rel_weight = 1
for batch_idx, return_dict in enumerate(train_lder):
    x = return_dict['poses_wrenches_actions_tensor'].to(device)
    y = return_dict['target'].float().to(device)
    print(y)
    # Forward propogation happens here
    output = model(x).to(device) 
    print(output)
    train_loss, pos_loss_ratio = GNLL_loss(output[:, :5], output[:, 5:], y, ori_rel_weight, log_pos_ratio=True)
    print(train_loss)
    print(pos_loss_ratio)
    break

tensor([[ 0.5596,  0.0032,  0.5035, -0.2501,  0.0369],
        [ 0.5500, -0.0113,  0.5035,  0.0594,  0.0219]], device='cuda:0')
tensor([[-0.1676, -0.1707, -0.1165, -0.3216, -0.0952,  1.1079,  1.1056,  1.2453,
          0.7929,  1.2192],
        [-0.1875, -0.1855, -0.0759, -0.2852, -0.0658,  1.1472,  1.0395,  1.2181,
          0.8046,  1.1991]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor(5.2179, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6427, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
def test_metrics(model, ori_rel_weight, seq_length, val_loader): #TODO add early stopping criterion
    # logging_step = 0
    # quantiles of interest: median and 95% CI
    q = torch.as_tensor([0.025, 0.5, 0.975]).to(device) 
    q_timing = torch.as_tensor([0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).to(device)
    seq_length = seq_length
    ## switch model to eval
    model.eval()

    with torch.no_grad():
        for t in range(seq_length):

            total_valid_pos_error, total_valid_ori_error, total_valid_loss = [], [], []
            for batch_idx,(x,y, t) in enumerate(val_loader):
                x = x.to(device)
                y = y.float().to(device)

                # Forward propogation happens here
                outputs = model(x).to(device) 
                output_t = outputs[:, t, :]

                loss = weighted_MSE_loss(output_t, y, ori_rel_weight)

                ## evaluate and append analysis metrics
                total_valid_ori_error.append(batched_ori_err(output_t, y))
                total_valid_pos_error.append(batched_pos_err(output_t, y))
                total_valid_loss.append(loss)

                # if batch_idx % log_interval == 0:
                #     wandb.log({"loss": loss, 'epoch': epoch, 'batch_idx': batch_idx})
                
                

            total_valid_pos_error = torch.cat(total_valid_pos_error).to(device)
            total_valid_ori_error = torch.cat(total_valid_ori_error).to(device)
            total_valid_loss = torch.as_tensor(total_valid_loss).to(device)

            ## statistical metrics from the test evaluations

            ## pos error
            pos_err_mean = torch.mean(total_valid_pos_error)
            pos_err_std = torch.std(total_valid_pos_error)
            pos_err_max = torch.max(total_valid_pos_error)
            pos_err_min = torch.min(total_valid_pos_error)

            ## 95% confidence interval and median
            # q = torch.as_tensor([0.025, 0.5, 0.975]) 
            pos_err_95_median = torch.quantile(total_valid_pos_error, q, dim=0, keepdim=False, interpolation='nearest')

            ## ori error
            ori_err_mean = torch.mean(total_valid_ori_error)
            ori_err_std = torch.std(total_valid_ori_error)
            ori_err_max = torch.max(total_valid_ori_error)
            ori_err_min = torch.min(total_valid_ori_error)

            ## 95% confidence interval
            ori_err_95_median = torch.quantile(total_valid_ori_error, q, dim=0, keepdim=False, interpolation='nearest')

            ## loss 
            loss_mean = torch.mean(total_valid_loss)
            loss_std = torch.std(total_valid_loss)
            loss_max = torch.max(total_valid_loss)
            loss_min = torch.min(total_valid_loss)

            ## 95% confidence interval
            loss_95_median = torch.quantile(total_valid_loss, q, dim=0, keepdim=False, interpolation='nearest')

            wandb.log({ 
            'valid_pos_err_mean_' + str(t) : pos_err_mean,
            'valid_pos_err_std_' + str(t) : pos_err_std,
            'valid_pos_err_max_' + str(t) : pos_err_max,
            'valid_pos_err_min_' + str(t) : pos_err_min,
            'valid_pos_err_95_lower_' + str(t) : pos_err_95_median[0].item(),
            'valid_pos_err_median_' + str(t) : pos_err_95_median[1].item(),
            'valid_pos_err_95_upper_' + str(t) : pos_err_95_median[2].item(),
            'valid_ori_err_mean_' + str(t) : ori_err_mean,
            'valid_ori_err_std_' + str(t) : ori_err_std,
            'valid_ori_err_max_' + str(t) : ori_err_max,
            'valid_ori_err_min_' + str(t) : ori_err_min,
            'valid_ori_err_95_lower_' + str(t) : ori_err_95_median[0].item(),
            'valid_ori_err_median_' + str(t) : ori_err_95_median[1].item(),
            'valid_ori_err_95_upper_' + str(t) : ori_err_95_median[2].item(),
            'valid_loss_mean_' + str(t) : loss_mean,
            'valid_loss_std_' + str(t) : loss_std,
            'valid_loss_max_' + str(t) : loss_max,
            'valid_loss_min_' + str(t) : loss_min,
            'valid_loss_95_lower_' + str(t) : loss_95_median[0].item(),
            'valid_loss_median_' + str(t) : loss_95_median[1].item(),
            'valid_loss_95_upper_' + str(t) : loss_95_median[2].item()
            }, step = logging_step-1)
            ## log some summary metrics from the validation/eval run

            ## log a figure of model output  

        ## switch model back to train
        model.train()
             
    return model 

In [138]:
def training_loop(net,optimizer,criterion,num_epochs,train_loader):
  overall_step = 0
  counter = 0
  loss_plot = []
  for epoch in range(num_epochs):
    for i,(x,y) in enumerate(train_loader):
      counter += 1
      optimizer.zero_grad()

      x = x.to(device)
      y = y.float().to(device)

      # Forward propogation happens here
      outputs = net(x).to(device) 

      # The following lines of code is where the magic happens!
      # The partial derivatives of the Loss with respect to the 
      # weights are calculated and the weights are updated
      loss = criterion(outputs,y)  
      loss.backward()
      optimizer.step()
      loss_plot.append(loss.item())

      # if (counter+1) % 10 == 0: 
          # argmax = torch.max(outputs, 1)[1]
          # accuracy = (y == argmax.squeeze()).float().mean()

#           info = {'loss' : loss.item()}
#           for tag, value in info.items():
#               logger.scalar_summary(tag, value, overall_step)
#           overall_step+=1

#           print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
#                  %(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
  return net #, loss_plot

In [216]:
num_epochs = 10
learning_rate = 0.003

# net = My_Neural_Network()
# net = net.to(device)

# model = ScrewingModel(input_dim, hidden_dim, ouput_dim) # no specification of 
model = model.to(device)
loss_function = nn.MSELoss(reduction='sum') #alternatively mean the squared error or none ...
optimizer = optim.Adam(model.parameters(), lr=learning_rate)




In [217]:
a = torch.rand(2,5)
print(a)
print(torch.norm(a, 2, 1))
ones = torch.ones(2, 1)
print(ones)
torch.cat((a, ones), 1)

b  = torch.rand(2,5)
print(b)
a / b
# a.size()[0]
# len(a)

tensor([[0.4243, 0.1488, 0.7550, 0.0404, 0.7973],
        [0.7016, 0.0033, 0.2402, 0.8475, 0.3636]])
tensor([1.1872, 1.1833])
tensor([[1.],
        [1.]])
tensor([[0.9657, 0.3413, 0.5083, 0.4488, 0.7348],
        [0.2556, 0.4523, 0.4474, 0.1619, 0.5287]])


tensor([[0.4394, 0.4359, 1.4852, 0.0901, 1.0850],
        [2.7447, 0.0073, 0.5368, 5.2353, 0.6876]])

In [98]:
def weighted_MSE_loss(input, target, ori_weight):
    SE_out = (input - target)**2
    # print(torch.mean(SE_out))
    # print(torch.sum(SE_out)/torch.numel(input))
    pos_SE = SE_out[:, :3]
    ori_SE = SE_out[:, 3:]
    # print((torch.sum(pos_SE) + torch.sum(ori_SE))/torch.numel(input))
    weighted_mean = (torch.sum(pos_SE) + ori_weight*torch.sum(ori_SE))/len(SE_out)
    # weighted_sum = torch.sum(pos_SE) + ori_weight*torch.sum(ori_SE)
    return weighted_mean
    # return torch.sum(torch.mul(weighting, MSE_out))
    # return torch.sum(MSE_out)

In [151]:
with torch.no_grad():
    for i, (x, y) in enumerate(train_lder):
        # print(x.size())
        x = x.to(device)
        out = model(x)
        print(out)
        y = y.float().to(device)
        print(y)
        # print(torch.bmm(out, y)) 
        bvv = torch.bmm(out.view(2, 1, 5), y.view(2, 5, 1)).reshape(-1)
        print(bvv)
        print(bvv.size())
        # print((out**2).sum(1, keepdim=True))
        SE_out = torch.sqrt((out**2).sum(1, keepdim=False))
        print(SE_out)
        print(SE_out.size())
        # print(out.size()[1])
        
        print(torch.cat((bvv, SE_out)))
        
        list_batch = [bvv, SE_out]
        list_batch = torch.cat(list_batch)
        print(list_batch)
        
        ori_weighting = 5 # relative to position weighting
        MSE = nn.MSELoss(reduction='none')
        # print(weighted_MSE_loss(out, y, ori_weighting))
        break
    

tensor([[ 0.0254, -0.1393,  0.1538, -0.1316,  0.0819],
        [ 0.0253, -0.1405,  0.1519, -0.1305,  0.0827]], device='cuda:0')
tensor([[ 0.5526, -0.0125,  0.5124, -0.0292, -0.0315],
        [ 0.5529, -0.0057,  0.5125, -0.0111, -0.0009]], device='cuda:0')
tensor([0.0959, 0.0940], device='cuda:0')
torch.Size([2])
tensor([0.2603, 0.2594], device='cuda:0')
torch.Size([2])
tensor([0.0959, 0.0940, 0.2603, 0.2594], device='cuda:0')
tensor([0.0959, 0.0940, 0.2603, 0.2594], device='cuda:0')


In [ ]:
trained_model = training_loop(model,optimizer,loss_function,num_epochs,train_lder)


In [155]:
with torch.no_grad():
    for i, (x, y) in enumerate(train_lder):
        # print(x.size())
        x = x.to(device)
        out = trained_model(x)
        y = y.to(device)
        # out = out.to(device)
        print(loss_function(out, y))
        print(out)
        print(y)
        break

tensor(0.0587, device='cuda:0', dtype=torch.float64)
tensor([[ 0.5505, -0.0080,  0.5124, -0.0217, -0.0615],
        [ 0.5471,  0.0051,  0.5139, -0.0379,  0.0237]], device='cuda:0')
tensor([[ 0.5764, -0.0043,  0.5101,  0.2047,  0.0008],
        [ 0.5393,  0.0024,  0.5119, -0.0801,  0.0556]], device='cuda:0',
       dtype=torch.float64)


In [158]:
out

tensor([[ 0.5591, -0.0023,  0.5127,  0.0754, -0.0207],
        [ 0.5488,  0.0055,  0.5150, -0.0272,  0.0273]], device='cuda:0')

In [179]:
# model_save_dir = os.getcwd()
model_save_dir = '../../models'
model_name = time.strftime("/model_%Y-%m-%d_%H-%M-%S.pt")
# model_name = '/test_model.pt'
torch.save(trained_model.state_dict(), model_save_dir + model_name)

In [178]:
torch.save(trained_model.state_dict(), model_save_dir + model_name)


In [173]:
load_model = ScrewingModel(input_dim, hidden_dim, num_layers, output_dim)
load_model.load_state_dict(torch.load(model_save_dir + model_name))
load_model.eval()

ScrewingModel(
  (lstm): LSTM(19, 38, num_layers=5, batch_first=True)
  (hidden2out): Linear(in_features=38, out_features=5, bias=True)
)